In [12]:
# Step 1: Import required libraries
import requests
import os
from datetime import datetime
from email.mime.text import MIMEText
import smtplib



In [13]:
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",     
    "ids": "bitcoin",         
    "price_change_percentage": "24h,7d",  
    "per_page": 1,
    "page": 1,
    "sparkline": False
}

r = requests.get(url, params=params)
print(r.status_code)   # return 200
print(r.json())        # see response


200
[{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin', 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400', 'current_price': 111421, 'market_cap': 2214395543348, 'market_cap_rank': 1, 'fully_diluted_valuation': 2214397322652, 'total_volume': 36622772880, 'high_24h': 112555, 'low_24h': 110464, 'price_change_24h': -242.98584973432298, 'price_change_percentage_24h': -0.2176, 'market_cap_change_24h': -12630152428.092773, 'market_cap_change_percentage_24h': -0.56713, 'circulating_supply': 19912459.0, 'total_supply': 19912475.0, 'max_supply': 21000000.0, 'ath': 124128, 'ath_change_percentage': -10.38507, 'ath_date': '2025-08-14T00:37:02.582Z', 'atl': 67.81, 'atl_change_percentage': 163944.9849, 'atl_date': '2013-07-06T00:00:00.000Z', 'roi': None, 'last_updated': '2025-08-28T00:12:00.665Z', 'price_change_percentage_24h_in_currency': -0.2176038009332798, 'price_change_percentage_7d_in_currency': -2.4762066928386015}]


In [14]:
r = requests.get(url, params=params)

r

<Response [200]>

In [15]:
r = requests.get(url, params=params)
print(type(r.text))
print(r.text)

<class 'str'>
[{"id":"bitcoin","symbol":"btc","name":"Bitcoin","image":"https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400","current_price":111421,"market_cap":2214395543348,"market_cap_rank":1,"fully_diluted_valuation":2214397322652,"total_volume":36622772880,"high_24h":112555,"low_24h":110464,"price_change_24h":-242.98584973432298,"price_change_percentage_24h":-0.2176,"market_cap_change_24h":-12630152428.092773,"market_cap_change_percentage_24h":-0.56713,"circulating_supply":19912459.0,"total_supply":19912475.0,"max_supply":21000000.0,"ath":124128,"ath_change_percentage":-10.38507,"ath_date":"2025-08-14T00:37:02.582Z","atl":67.81,"atl_change_percentage":163944.9849,"atl_date":"2013-07-06T00:00:00.000Z","roi":null,"last_updated":"2025-08-28T00:12:00.665Z","price_change_percentage_24h_in_currency":-0.2176038009332798,"price_change_percentage_7d_in_currency":-2.4762066928386015}]


In [16]:
print(r.encoding)
print(r.status_code)

utf-8
200


In [118]:
def get_btc_markets():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",     
        "ids": "bitcoin",         
        "price_change_percentage": "24h,7d",  
        "per_page": 1,
        "page": 1,
        "sparkline": False
    }
    headers = {"accept": "application/json", "User-Agent": "Mozilla/5.0"}
    r = requests.get(url, params=params, headers=headers, timeout=20)
    r.raise_for_status()
    data = r.json()
    if not data:
        raise RuntimeError("Empty markets response")
    item = data[0]  # list → first object 
    return {
        "price": item["current_price"],
        "pct_24h": item.get("price_change_percentage_24h_in_currency"),
        "pct_7d": item.get("price_change_percentage_7d_in_currency")
    }


In [3]:
def send_alert(message):
    print(f"[{datetime.now()}] ALERT: {message}")

In [8]:
EMAIL_SENDER = os.environ.get("EMAIL_SENDER")
EMAIL_RECEIVER = os.environ.get("EMAIL_RECEIVER")
EMAIL_PASSWORD = os.environ.get("EMAIL_PASSWORD")  # Gmail App Password

# =========================
#  Fetch BTC Market Data
# =========================
def get_btc_markets():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "ids": "bitcoin",
        "price_change_percentage": "24h,7d",
        "per_page": 1,
        "page": 1,
        "sparkline": False
    }
    headers = {"accept": "application/json", "User-Agent": "Mozilla/5.0"}
    r = requests.get(url, params=params, headers=headers, timeout=20)
    r.raise_for_status()
    data = r.json()
    if not data:
        raise RuntimeError("Empty markets response")
    item = data[0]
    return {
        "price": item["current_price"],
        "pct_24h": item.get("price_change_percentage_24h_in_currency"),
        "pct_7d": item.get("price_change_percentage_7d_in_currency")
    }

# =========================
#  Send Email Alert
# =========================
def send_email_alert(subject, message):
    if not (EMAIL_SENDER and EMAIL_RECEIVER and EMAIL_PASSWORD):
        print("❌ Missing environment variables for email config")
        return
    
    try:
        msg = MIMEText(message)
        msg["Subject"] = subject
        msg["From"] = EMAIL_SENDER
        msg["To"] = EMAIL_RECEIVER

        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.sendmail(EMAIL_SENDER, EMAIL_RECEIVER, msg.as_string())
        
        print(f"[{datetime.now()}] ✅ Email alert sent")
    except Exception as e:
        print(f"[{datetime.now()}] ❌ Email failed: {e}")

# =========================
#  Check thresholds + trigger alerts
# =========================
def check_alerts():
    btc = get_btc_markets()
    price = btc["price"]
    pct_24h = btc["pct_24h"]
    pct_7d = btc["pct_7d"]

    print(f"[{datetime.now()}] BTC ${price} | 24h: {pct_24h:.2f}% | 7d: {pct_7d:.2f}%")

    alerts = []

    # Example rules — adjust thresholds as needed
    if pct_24h is not None and pct_24h <= -10:
        alerts.append(f"⚠️ BTC down {pct_24h:.2f}% in last 24h! Current price: ${price}")
    if pct_7d is not None and pct_7d <= -20:
        alerts.append(f"🚨 BTC down {pct_7d:.2f}% in last 7 days! Current price: ${price}")

    if alerts:
        send_email_alert("BTC Market Alert", "\n".join(alerts))

# =========================
#  Main
# =========================
if __name__ == "__main__":
    check_alerts()

[2025-08-27 19:42:57.584985] BTC $111213 | 24h: -0.54% | 7d: -2.95%


In [9]:
EMAIL_SENDER = "onyekachi.amaefule031@gmail.com"
EMAIL_RECEIVER = "jessenia@datainbloom.dev"
EMAIL_PASSWORD = "your_app_password"